In [3]:
import torch
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
import os

In [4]:
def displayIMG(path):
    from ipywidgets import interact ,widgets
    return display(widgets.Image(value=open(path,'rb').read(),format='png',width=500,height=400,))

In [5]:
def get_action(max_loop,actions=['吃饭','睡觉','打豆豆','看电视','看小姐姐','等曹世同']):
    action_array , action_code_tensor = [],[]
    for _ in range(max_loop):
        np.random.seed(4)
        action_list = np.array(actions)#*np.random.randint(1,6))
        action_code = np.array([random.randint(0,len(action_list)-1) for i in  range(len(action_list))])#torch.torch.randperm(len(actions)).numpy()
        random_action = action_list[action_code]
        action_array.append(random_action)
        action_code_tensor.append(action_code)
    return np.array(action_array),torch.LongTensor(action_code_tensor)

In [6]:
actions , tensor = get_action(100)
actions

array([['等曹世同', '吃饭', '看电视', '看小姐姐', '等曹世同', '吃饭'],
       ['等曹世同', '看小姐姐', '等曹世同', '睡觉', '看电视', '看电视'],
       ['吃饭', '打豆豆', '等曹世同', '看电视', '睡觉', '看电视'],
       ['吃饭', '看电视', '看电视', '睡觉', '看电视', '看小姐姐'],
       ['吃饭', '看小姐姐', '睡觉', '睡觉', '吃饭', '看小姐姐'],
       ['等曹世同', '看电视', '看电视', '吃饭', '打豆豆', '吃饭'],
       ['打豆豆', '睡觉', '看小姐姐', '看小姐姐', '打豆豆', '看小姐姐'],
       ['看小姐姐', '吃饭', '看小姐姐', '吃饭', '睡觉', '吃饭'],
       ['等曹世同', '吃饭', '看电视', '看小姐姐', '等曹世同', '打豆豆'],
       ['打豆豆', '打豆豆', '看小姐姐', '吃饭', '看小姐姐', '打豆豆'],
       ['看小姐姐', '等曹世同', '吃饭', '等曹世同', '看小姐姐', '看小姐姐'],
       ['打豆豆', '看小姐姐', '吃饭', '看小姐姐', '吃饭', '看小姐姐'],
       ['打豆豆', '等曹世同', '等曹世同', '等曹世同', '看电视', '打豆豆'],
       ['睡觉', '睡觉', '睡觉', '睡觉', '吃饭', '睡觉'],
       ['等曹世同', '等曹世同', '吃饭', '睡觉', '看小姐姐', '看小姐姐'],
       ['看电视', '看小姐姐', '吃饭', '睡觉', '等曹世同', '睡觉'],
       ['看电视', '睡觉', '打豆豆', '看小姐姐', '看小姐姐', '打豆豆'],
       ['看小姐姐', '吃饭', '看电视', '睡觉', '看电视', '看电视'],
       ['打豆豆', '睡觉', '等曹世同', '看小姐姐', '打豆豆', '吃饭'],
       ['看小姐姐', '打豆豆', '打豆豆

## 1.第一个概念是⻢尔科夫性
### 所谓⻢尔科夫性是指系统的下一个状态 $st+1$ 仅与当前状态 $st$ 有关，而 与以前的状态无关。
### $ 定义:状态 $ st $ 是⻢尔科夫的，当且仅当
## $$ P[st+1 |st ]=P[st+1 |s1 ，...，st ]$$
### 定义中可以看到，当前状态st 其实是蕴含了所有相关的历史信息s1 ， ...，st ，一旦当前状态已知，历史信息将会被抛弃。
### ⻢尔科夫性描述的是每个状态的性质，但真正有用的是如何描述一个 状态序列。数学中用来描述随机变量序列的学科叫随机过程。
### 所谓随机过 程就是指随机变量序列。若随机变量序列中的每个状态都是⻢尔科夫的， 则称此随机过程为⻢尔科夫随机过程。

## 2.第二个概念是⻢尔科夫过程 ⻢尔科夫过程的定义:⻢尔科夫过程是一个二元组(S，P)，且满
### 足:S是有限状态集合，P是状态转移概率。状态转移概率矩阵为: 
### 。下面我们以一个例子来进行阐述。
### 如图2.2所示为一个学生的7种状态
$$ {娱乐，课程1，课程2，课程3，考 过，睡觉，论文} $$，
### 每种状态之间的转换概率如图所示。则该生从课程 1 开 始一天可能的状态序列为:
$$ {课1-课2-课3-考过-睡觉} $$
$$ {课1-课2-睡觉} $$
### 以上状态序列称为⻢尔科夫链。当给定状态转移概率时，从某个状态
### 出发存在多条⻢尔科夫链。对于游戏或者机器人，⻢尔科夫过程不足以描
### 述其特点，因为不管是游戏还是机器人，他们都是通过动作与环境进行交
<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/4BC095DB-D70E-4A36-AD82-77258617B7BD.png">

## 马尔科夫状态转移  <代码示例>

In [7]:
def transition_probability(A):
    left , right = [],[]
    initA = np.array(A)
    An = initA.shape[0]
    unitVector = np.ones(An)
    distionA = np.array([j for j in {i:0 for i in A}.keys()])
    probability = lambda  elements , n : {e:(elements==e).dot(np.ones(n))/n for e in elements}
    prob_list = lambda prob_dict:[prob_dict[i] for i in prob_dict]

    transfer_matrix_left , transfer_matrix_right,nameList_left,nameList_right = [],[],[],[]
    for e in distionA:
        one_E_index = np.where(initA==e)[0]
        bool_left = one_E_index[one_E_index -1 >= 0]
        bool_right = one_E_index[one_E_index + 1 < An]
        left = bool_left - 1
        right = bool_right + 1
        if len(left) > 0:
            nameList_left.append(e)
            init_dict_left = {i : 0 for i in initA}
            left_elements = initA[left]
            ln = left_elements.shape[0]
            prob_left = probability(left_elements,ln)
            init_dict_left.update(prob_left)
            transfer_matrix_left.append(prob_list(init_dict_left))
            #print('left',e,transfer_matrix_left[-1],init_dict_left)
        if len(right) > 0:
            nameList_right.append(e)
            init_dict_right = {i : 0 for i in initA}
            right_elements = initA[right]
            rn = right_elements.shape[0]
            prob_right = probability(right_elements,rn)
            init_dict_right.update(prob_right)
            transfer_matrix_right.append(prob_list(init_dict_right))
            #print('right',e,transfer_matrix_right[-1],init_dict_right)
         
    return {'cloumn_names_left':init_dict_left.keys()
    ,'transfer_matrix_left':np.array(transfer_matrix_left)
    ,'cloumn_names_right':init_dict_right.keys()
    ,'transfer_matrix_right':np.array(transfer_matrix_right)}

In [8]:
tp = transition_probability(np.ravel(actions))

In [9]:
def markov(init_array,transfer_matrix,maxloop=100):
    #转移矩阵
    results = []
    restmp = init_array #迭代结果
    for i in range(maxloop):
        res = np.dot(transfer_matrix.T,restmp)
        results.append(res)
        restmp = res
    return np.array(results)



In [10]:
column = tp['cloumn_names_right']
init_array = np.ones(len(column))/len(column)
transfer_matrix = tp['transfer_matrix_right']
init_array,transfer_matrix 

(array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667]),
 array([[0.14736842, 0.25263158, 0.16842105, 0.13684211, 0.12631579,
         0.16842105],
        [0.12380952, 0.17142857, 0.14285714, 0.19047619, 0.2       ,
         0.17142857],
        [0.13953488, 0.13953488, 0.12790698, 0.1744186 , 0.19767442,
         0.22093023],
        [0.18181818, 0.21212121, 0.07070707, 0.13131313, 0.13131313,
         0.27272727],
        [0.17346939, 0.18367347, 0.15306122, 0.18367347, 0.17346939,
         0.13265306],
        [0.17241379, 0.10344828, 0.18965517, 0.17241379, 0.15517241,
         0.20689655]]))

In [11]:
print(tp['cloumn_names_right'])
%time markov(init_array,transfer_matrix,1000000)

dict_keys(['等曹世同', '吃饭', '看电视', '看小姐姐', '睡觉', '打豆豆'])
CPU times: user 2 s, sys: 115 ms, total: 2.12 s
Wall time: 2.18 s


array([[0.15640237, 0.17713967, 0.14210144, 0.16485622, 0.16399086,
        0.19550946],
       [0.15693823, 0.17502247, 0.1436594 , 0.16540554, 0.16370669,
        0.19526766],
       [0.15698136, 0.17505161, 0.14359596, 0.16532558, 0.16364423,
        0.19540126],
       ...,
       [0.15698013, 0.17504379, 0.1436089 , 0.16532688, 0.16364254,
        0.19539777],
       [0.15698013, 0.17504379, 0.1436089 , 0.16532688, 0.16364254,
        0.19539777],
       [0.15698013, 0.17504379, 0.1436089 , 0.16532688, 0.16364254,
        0.19539777]])

# 马尔科夫决策过程
## 跟⻢尔科夫过程不同的是，⻢尔科夫决策过程的状态转移概率是包含动作的，即 

In [12]:
displayIMG("/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/CBAA3E25-066E-40E1-86D6-119DF3008A85.png")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00^Exif\x00\x00MM\x00*\x…

# $$ S={s1 ，s2 ，s3 ，s4 ，s5 } <状态集>$$
## $$ A= {玩，退出，学习，发表，睡觉} <动作集> $$
## $$ R <立即汇报> $$
## ⻢尔科夫决策过程由元组
## $$(S，A，P，R，γ) $$ 
## 描述，其中: $ S $ 为有限的状态集
## A 为有限的动作集
## P 为状态转移概率
## R 为回报函数
## γ 为折扣因子，用来计算累积回报。
## 跟⻢尔科夫过程不同的是，⻢尔科夫决策过程的状态转移概率是包含动作的，即 : 
## $$ P^{a}_{ss} = P[S_{t+1} = s' | S_{t} = s , A_t = a] $$
## &- 公式解释:
### 当前动作 $ P^{a}_{ss} 的概率 = (S_{t+1} 时刻 s' 的概率) \cap ({S_{t} = s \cup A_t = a }时刻的概率）$
## -&
### 强化学习的目标是给定一个⻢尔科夫决策过程，寻找最优策略。所谓 策略是指状态到动作的映射，策略常用符号π表示，它是指给定状态s时， 动作集上的一个分布，即：
## $$ \pi(a|s) = p[ A_t = a|S_t = s] $$
## &- 公式解释:
### 策略π在每个$状态s$指定一个动作概率。如果给出的策略π是确定性的，那么策略π在每个状态s指定一个确定的动作。
## -&

### 强化学习的策略往往是随机策略。采用随机策略的好处是可以将探索耦合到采样的过程中。所谓探索是指机器人尝试其他的动作便找到更好的策略。其次，在实际应用中，存在各种噪声，这些噪声大都服从正态分布，如何去掉这些噪声也需要用到概率的知识。

In [13]:
displayIMG("/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/10791425-712E-4071-BBCF-7C83ECD2CE28.png")


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00^Exif\x00\x00MM\x00*\x…

## 学生的策略为π1 (玩|s1 )=0.8，是指该学生在状态s1 时玩的概率为0.8，不玩的概率是0.2，显然这个学生更喜欢玩。
## 另外一个学生的策略为π2 (玩|s1 )=0.3，是指该学生在状态s1 时玩 的概率是 0.3，显然这个学生不爱玩。依此类推，每个学## 生都有自己的策
## 略。强化学习是找到最优的策略，这里的最优是指得到的总回报最大。


In [14]:
displayIMG("/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/0D20A223-BC1A-4A01-B00D-58888D199254.png")
print("""
S  为有限的状态集
A 为有限的动作集
P 为状态转移概率
R 为回报函数
γ 为折扣因子，用来计算累积回报。
""")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00^Exif\x00\x00MM\x00*\x…


S  为有限的状态集
A 为有限的动作集
P 为状态转移概率
R 为回报函数
γ 为折扣因子，用来计算累积回报。



In [15]:
## 创建随机历史动作和动作编码
A , Acode = get_action(100,["玩儿","睡觉","学习","发表",'空闲','空闲'])

In [ ]:
## 计算历史动作概率
def game_start_prob(A):
    Phistory = transition_probability(np.ravel(A))
    mat_p = Phistory['transfer_matrix_right']
    actions_list = Phistory['cloumn_names_right']
    init_start = np.ones(mat_P.shape[0])/matP.shape[0]
    return markov(init_start,matP)[-1],actions_list
init_p , actions_list = game_start_prob(A)
init_p,actions_list

In [17]:
def G(R,Alpha):
    pass

In [18]:
def epsilon_pi():
    pass

## $$ v_{\pi}(s) \doteq \mathbb{E}_{\pi}[G_{t}|S_{t}=s] = \mathbb{E}_{\pi}\left [ \sum_{k=0}^{\infty}\gamma^{k}R_{t+k+1}\mid S_{t}=s \right ], \ for \ all \ s\in S $$


## $$ Q_{\pi}(s,a) \doteq \mathbb{E}_{\pi}[G_{t}|S_{t}=s,A_{t}=a] = \mathbb{E}_{\pi}\left [ \sum_{k=0}^{\infty}\gamma^{k}R_{t+k+1}\mid S_{t}=s,A_{t}=a \right ] $$

from torchnlp.word_to_vector import GloVe
vector = GloVe()
vectors['hello']